In [2]:
import os
import shutil
import csv
import numpy as np
from PIL import Image
from tabulate import tabulate

In [1]:


# In questo notebook utilizzerò i path relativi in modo tale che non importa dove si trovi la cartella
# "files", l'importante è che le sottocartelle e i file siano tutti contenuti nella cartella "files"
files_list = os.listdir("files")  # Creo una lista dei file e cartelle contenuti nella directory "files"

# Creo un dizionario per identificare di che tipo sono le estensioni associandole a delle chiavi in 
# base al tipo
extensions = {
    "Images": [".jpg", ".jpeg", ".png"],
    "Docs": [".doc", ".txt", ".odt"],
    "Audio": [".mp3"]
}

# Apro il csv, la parola chiave "with", mi permette di chiudere automaticamente il file senza doverlo 
# specificare, gli argomenti della funzione open, sono rispettivamente il path dove voglio che 
# si posizionerà il csv; la modalità: in questo caso "a" per append, in modo da non sovrascrivere il 
# csv ogni volta che viene eseguito lo script, ma solo aggiornandolo in ordine; l' encoding nel caso 
# ci siano dei caratteri speciali, new line per evitare che ci sia una riga vuota. "f" sarà il nome 
# con cui mi riferirò all' handler
with open("files/recap.csv", "a", encoding="UTF8", newline="") as f:

    # Creo un writer utilizzando la classe del modulo csv DictWriter, grazie al quale insieme ad un 
    # dizionario scriverò le informazioni nel csv. "fieldnames" saranno i campi del mio header.
    writer = csv.DictWriter(f, fieldnames=["name", "type", "size(B)"])
    if f.tell() == 0:  # Controllo se il file csv è vuoto o meno
        writer.writeheader()  # Se è vuoto, scrivo un header
    
    # Ciclo nel dizionario delle estensioni, di default il valore assunto è la chiave                                               
    for key in extensions:  

        # Controllo che esista una directory per i tipi di estensione supportati dallo script,
        # se non c'è, la creo.
            if not os.path.isdir(f"files/{key}"):
                os.makedirs(f"files/{key}")
        
            for file in files_list:  # Ciclo la lista  dei file ordinata ottenuta con listdir()
                # Controllo se il valore assunto dalla variabile è un file o meno
                if os.path.isfile(f"files/{file}"):  
                    # Isolo l' estensione e il nome del file in due variabili                                
                    file_name, extension = os.path.splitext(file)  
                        # Controllo che l'estensione del file sia supportata
                        if extension in extensions[key]:     
                            src = f"files/{file}"  # Creo una variabile con il path di origine
                            dst = f"files/{key}/{file}"  # E una con il path di destinazione
                            shutil.move(src=src, dst=dst)  # Sposto il file da origine a destinazione

                            # Creo una lista contenente un dizionario con le informazioni da inserire
                            # nel csv, mi serve un iterabile da passare al writer.writerows
                            info = [{
                                "name": file_name,
                                "type": key,
                                "size(B)": os.path.getsize(dst)
                            }]
                            writer.writerows(info)  # Inserisco i dati del file spostato nel csv
                            print(f"{info[0]['name']} type: {info[0]['type']} size: {info[0]['size(B)']}B")  
                        # Se non è supportata lo comunico all'utente
                        else:
                            print(f"the extension {extension} is not yet supported")













bw.png type: Images size: 94926B
ciao.txt type: Docs size: 12B
daffodil.jpg type: Images size: 24657B
eclipse.png type: Images size: 64243B
pippo.odt type: Docs size: 8299B
song1.mp3 type: Audio size: 1087849B
song2.mp3 type: Audio size: 764176B
trump.jpeg type: Images size: 10195B


In [4]:


# Creo una lista dei file contenuti nella cartella docs
images_list = os.listdir("files/Images")

# Creo un dizionario con le informazioni da inserire nella tabella, le chiavi saranno l'header
# I valori saranno una lista che verrà processata da tabulate assegnandoli rispettivamente ad ogni 
# File processato
table = {
        "name": [],
        "height": [],
        "width": [],
        "grayscale" : [],
        "R" : [],
        "G" : [],
        "B" : [],
        "ALPHA" : []   
    }

# Ciclo i filenella cartella, ottengo il path le processo ad Image.open, che mi creerà un oggetto Image

for file in images_list:
    path = f"files/Images/{file}"
    img = Image.open(path)
    
    # Assegno i valori alla chiave "name" tramite os.path.splitext che mi isolerà il nome del file 
    # Dall' estensione. Ottengo una lista contenente ad indice 0 il nome del file, cui accedo con uno
    # Slicing di lista
    table["name"].append(os.path.splitext(file)[0])
    
    # Creo il mio array con la funzione np.array e come argomento l'oggetto Image ottenuto prima
    np_img = np.array(img)
    
    # Assegno i valori "height" and "width" ottenuti con array.shape, che mi restituisce in numero di 
    # righe e colonne dell'array, ovvero il numero di pixel in altezza e larghezza.
    # array.shape mi restituisce una tupla, accedo quindi con uno slicing ai rispettivi valori
    table["height"].append(np_img.shape[0])
    table["width"].append(np_img.shape[1])
    
    # Le immagini possono essere in scala di grigio, rgb, o rbga. Un immagine in scala di grigio avrà 
    # Una dimensione pari a 2 (matrice 2x2), una rgb e una rgba avrà dimensione 3 (matrice 2x2 dove 
    # Ogni valore sarà una tupla (x, y, z), quindi isolando avremo 3 matrici 2x2 dove ognuna
    # Eappresenterà rispettivamente il rosso, il verde, e il blu). Stesso discorso per l'immagine con
    # Il canale alpha, ma in questo caso avremo 4 matrici 2x2, dove la quarta avrà appunto i valori 
    # Alpha. Prima di tutto cerco di capire se il file sia un' immagine in scala di grigio o meno
    # Tramite un if statement controllando la dimensione dell'array
    if np_img.ndim < 3:
        # Assegno quindi alla chiave "grayscale" il valore medio ottenuto dai pixel di tutte le righe
        # E tutte le colonne (:,:) tramite uno slicing
        table["grayscale"].append(np.mean(np_img))
        # Non essendoci altri canali, tutti gli altri valori avranno valore 0
        for key in ["R", "G", "B", "ALPHA"]:
            table[key].append(0)
    else:
        # Nel caso opposto, se la dimensione è maggiore di 2, grayscale sarà uguale a 0
        table["grayscale"].append(0)
        
        # Per quanto riguarda le immagini a colori, calcolo la media usando il parametro "axis=(0,1)".
        # In questo modo sto calcolando la media facendo collassare le righe (axis=0) e le colonne
        # (axis=1), ottenendo una tupla (x, y, z) dove ogni valore è rispettivamente la media 
        # per livello di colore.
        mean_tup = np.mean(np_img, axis=(0,1))
        # assegno poi alla tabella i rispettivi valori medi
        for n, key in enumerate(["R", "G", "B"]):
            table[key].append(mean_tup[n])
        
        # Per l'immagina con canale alpha, utilizzo la stessa variabile "mean_tup", che mi restituirà
        # una tupla (x, y, z, w). Il concetto poi si ripete come sopra
        if np_img.shape[2] == 4:
            table["ALPHA"].append(mean_tup[3])
        else:
            # In alternativa il valore medio di alpha sarà pari a 0
            table["ALPHA"].append(0)
            
# Stampo le informazioni ottenuto in una tabella creata con il modulo tabulate
# Table è il nome del dizionario creato ad inizio script, i suoi elementi (liste) vengono inseriti
# Ordinatamente in base alla chiave di appartenenza, che viene scritta come "Headers" assegnado 
# Dict.keys() al parametro "headers". "tablefmt" è semplicemente l'estetica della griglia. "floatfmt"
# Mi serve per arrotondare a due cifre decimali i valori da inserire.
# La funzione assume di default l'allineamento di testo a sinistra per le stringhe, e a destra per i 
# Numeri, inoltre converte automaticamente in numeri le stringhe che riconosce possano essere numeri. 
print(tabulate(table, headers=table.keys(), tablefmt="fancy_grid",floatfmt=".2f"))


    

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
